In [27]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json
import time

In [19]:
pip install seleniumbase

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.9/83.9 kB 808.8 kB/s eta 0:00:00a 0:00:01
  Using cached attrs-23.2.0-py3-none-any.whl.metadata (9.5 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 552.4/552.4 kB 2.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.9/147.9 kB 2.5 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.8/136.8 kB 2.4 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 kB 2.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.5/122.5 kB 2.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 2.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.0/116.0 kB 3.3 MB/s eta 0:00:00
   

In [25]:
pip install pyppeteer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 673.3 kB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 2.4 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install nest_asyncio


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [21]:
# Get the path for the ChromeDriver
driver_path = ChromeDriverManager().install()

# Set up Chrome options or capabilities (if needed)
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--disable-dev-shm-usage")


# Create a Chrome service with the driver path
chrome_service = Service(driver_path)

# Initialize the Chrome WebDriver with options and service
driver = webdriver.Chrome(service=chrome_service, options=chrome_options)

params = dict({
        "latitude": 22.6100,
        "longitude": 120.3500,
        "accuracy": 100
        })
driver.execute_cdp_cmd("Emulation.setGeolocationOverride", params)

try:
    driver.get("https://shop.ichefpos.com/explore")

    # Wait until the dialog box is visible
    wait = WebDriverWait(driver, 30)
    script = '''
        navigator.geolocation.getCurrentPosition(function(position) {
            console.log("Latitude: " + position.coords.latitude);
            console.log("Longitude: " + position.coords.longitude);
        });
    '''
    driver.execute_script(script)
    dialog_box = wait.until(EC.visibility_of_element_located((By.XPATH, "//div[@role='dialog']")))

    # Find the "Share Location" button inside the dialog box
    share_location_button = dialog_box.find_element(By.XPATH, ".//button[contains(., '分享位置')]")

    # Click the "Share Location" button
    share_location_button.click()

    # Wait until the dialog box is visible
    wait = WebDriverWait(driver, 30)
    # Execute JavaScript to get the geolocation coordinates
    script = '''
        navigator.geolocation.getCurrentPosition(function(position) {
            console.log("Latitude: " + position.coords.latitude);
            console.log("Longitude: " + position.coords.longitude);
        });
    '''
    driver.execute_script(script)

    # Wait for elements to load after clicking the button
    elements = wait.until(EC.visibility_of_all_elements_located((By.XPATH, "//*[contains(@id, 'restaurant-card')]")))

    shop_num = 0
    # Loop through each element to find anchor tags and extract href attribute
    for element in elements:
        anchor_tags = element.find_elements(By.TAG_NAME, 'a')
        for anchor_tag in anchor_tags:
            href = anchor_tag.get_attribute("href")
            print(href)
            shop_num += 1
    print(f'There are {shop_num} shops')

except Exception as e:
    print(f'An error occurred: {str(e)}')

finally:
    # Close the web driver
    driver.quit()


https://shop.ichefpos.com/store/MMt8M8MI/ordering?utm_source=safer_takeout&utm_medium=website&utm_campaign=safer_takeout&utm_id=safer_takeout_01
https://shop.ichefpos.com/store/Q4O89VAF/ordering?utm_source=safer_takeout&utm_medium=website&utm_campaign=safer_takeout&utm_id=safer_takeout_01
https://shop.ichefpos.com/store/RDD3AyCy/ordering?utm_source=safer_takeout&utm_medium=website&utm_campaign=safer_takeout&utm_id=safer_takeout_01
https://shop.ichefpos.com/store/TAjpiJrT/ordering?utm_source=safer_takeout&utm_medium=website&utm_campaign=safer_takeout&utm_id=safer_takeout_01
https://shop.ichefpos.com/store/g1NL9K9W/ordering?utm_source=safer_takeout&utm_medium=website&utm_campaign=safer_takeout&utm_id=safer_takeout_01
There are 5 shops


In [124]:
def setup_driver():
    # Get the path for the ChromeDriver
    driver_path = ChromeDriverManager().install()

    # Set up Chrome options or capabilities (if needed)
    chrome_options = webdriver.ChromeOptions()
    # chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--disable-dev-shm-usage")

    # Create a Chrome service with the driver path
    chrome_service = Service(driver_path)

    # Initialize the Chrome WebDriver with options and service
    driver = webdriver.Chrome(service=chrome_service, options=chrome_options)
    return driver

def scrape_restaurant_links(driver, latitude, longitude, first_open):
    try:
        params = {
            "latitude": latitude,
            "longitude": longitude,
            "accuracy": 100
        }
        driver.execute_cdp_cmd("Emulation.setGeolocationOverride", params)

        driver.get("https://shop.ichefpos.com/explore")

        if first_open:
            # Wait until the dialog box is visible
            wait = WebDriverWait(driver, 30)
            dialog_box = wait.until(EC.visibility_of_element_located((By.XPATH, "//div[@role='dialog']")))

            # Find the "Share Location" button inside the dialog box
            share_location_button = dialog_box.find_element(By.XPATH, ".//button[contains(., '分享位置')]")
            share_location_button.click()
        
        # Wait for the button to be clickable
        wait = WebDriverWait(driver, 10)
        button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".TakeoutFinderCircleButton__CircleButton-kxzgzF")))

        # Click the button
        button.click()

        wait = WebDriverWait(driver, 10)
        # Wait for elements to load after clicking the button
        elements = wait.until(EC.visibility_of_all_elements_located((By.XPATH, "//*[contains(@id, 'restaurant-card')]")))

        shop_links = []
        shop_ids = []
        # Loop through each element to find anchor tags and extract href attribute
        for element in elements:
            anchor_tags = element.find_elements(By.TAG_NAME, 'a')
            for anchor_tag in anchor_tags:
                href = anchor_tag.get_attribute("href")
                id = anchor_tag.get_attribute("id")
                shop_links.append(href)
                shop_ids.append(id)
        print(len(shop_ids), len(shop_links))
        return shop_links, shop_ids

    except Exception as e:
        print(f'An error occurred: {str(e)}')
        return [], []

In [111]:
def scrape_restaurant_links(latitude, longitude):
    # Get the path for the ChromeDriver
    driver_path = ChromeDriverManager().install()

    # Set up Chrome options or capabilities (if needed)
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--disable-dev-shm-usage")

    # Create a Chrome service with the driver path
    chrome_service = Service(driver_path)

    # Initialize the Chrome WebDriver with options and service
    driver = webdriver.Chrome(service=chrome_service, options=chrome_options)

    params = dict({
        "latitude": latitude,
        "longitude": longitude,
        "accuracy": 100
    })
    driver.execute_cdp_cmd("Emulation.setGeolocationOverride", params)

    try:
        driver.get("https://shop.ichefpos.com/explore")

        # Wait until the dialog box is visible
        wait = WebDriverWait(driver, 30)
 
        dialog_box = wait.until(EC.visibility_of_element_located((By.XPATH, "//div[@role='dialog']")))

        # Find the "Share Location" button inside the dialog box
        share_location_button = dialog_box.find_element(By.XPATH, ".//button[contains(., '分享位置')]")

        # Click the "Share Location" button
        share_location_button.click()

        # Wait until the dialog box is visible
        wait = WebDriverWait(driver, 30)

        # # Find the "AddIcon" button
        # add_button = wait.until(EC.visibility_of_element_located((By.XPATH, "//button[@data-testid='AddIcon']")))

        # # Click the "RemoveIcon" button
        # add_button.click()

        time.sleep(2)
        
        # Wait until the dialog box is visible
        wait = WebDriverWait(driver, 30)

        # Wait for elements to load after clicking the button
        # elements = wait.until(EC.visibility_of_all_elements_located((By.XPATH, "//*[@id='appPaper']/div[4]/div[1]/button[1]")))
        elements = wait.until(EC.visibility_of_all_elements_located((By.XPATH, "//*[contains(@id, 'restaurant-card')]")))


        shop_links = []
        shop_ids = []
        # Loop through each element to find anchor tags and extract href attribute
        for element in elements:
            anchor_tags = element.find_elements(By.TAG_NAME, 'a')
            for anchor_tag in anchor_tags:
                href = anchor_tag.get_attribute("href")
                id = anchor_tag.get_attribute("id")
                shop_links.append(href)
                shop_ids.append(id)
        print(len(shop_ids), len(shop_links))
        return shop_links, shop_ids

    except Exception as e:
        print(f'An error occurred: {str(e)}')
        return [], []

    finally:
        # Close the web driver
        driver.quit()



In [118]:
def store_restaurant_data(driver, latitude, longitude, first_open, filename='restaurant_data.json'):
    # Get the restaurant links and IDs
    restaurant_links, restaurant_ids = scrape_restaurant_links(driver, latitude, longitude, first_open)
    print(f'{len(restaurant_ids)} shops {len(restaurant_links)}')

    # Create a list of dictionaries to store the data
    restaurant_data = []
    for link, id in zip(restaurant_links, restaurant_ids):
        restaurant_data.append({'id': id, 'link': link})

    # Load existing data from the file (if any)
    try:
        with open(filename, 'r') as file:
            existing_data = json.load(file)
    except FileNotFoundError:
        existing_data = []

    # Append the new data to the existing data
    existing_data.extend(restaurant_data)

    # Write the updated data back to the file
    with open(filename, 'w') as file:
        json.dump(existing_data, file, indent=4)


In [97]:
# Taipei
north = 25.0500
west = 121.5000
east = 121.6000
south = 25.0100

step = 0.003

coords = []
current_lat = north
while current_lat >= south:
    current_lon = west
    while current_lon <= east:
        coords.append([current_lat, current_lon])
        current_lon += step
    current_lat -= step

In [98]:
len(coords)

476

In [133]:
coords = [[25.0447587,121.5451679]]

In [134]:
driver = setup_driver()
first_open = True
for coord in coords:
    lat = coord[0]
    long = coord[1]
    store_restaurant_data(driver, lat, long, first_open)
    first_open = False
    time.sleep(3)
driver.quit()

100 100
100 shops 100


In [18]:
import asyncio
from pyppeteer import launch

# Function to set geolocation using pyppeteer (Chrome DevTools Protocol)
async def set_geolocation(latitude, longitude):
    browser = await launch()
    page = await browser.newPage()

    # Evaluate JavaScript to set geolocation in the browser context
    await page.evaluateOnNewDocument(
        '''() => {
            navigator.geolocation.getCurrentPosition = function(success){
                const position = {coords: {latitude: %f, longitude: %f}};
                success(position);
            }
        }''' % (latitude, longitude)
    )

    # Navigate to the website
    await page.goto("https://shop.ichefpos.com/explore")

    # Wait until the dialog box is visible
    await page.waitForXPath("//div[@role='dialog']")

    # Find the "Share Location" button inside the dialog box
    share_location_button = await page.xpath("//button[contains(text(), '分享位置')]")

    # Click the "Share Location" button if found
    if share_location_button:
        await share_location_button[0].click()

    # Wait for 15 seconds
    await asyncio.sleep(30)

    # Wait for elements to load after clicking the button
    elements = await page.waitForXPath("//*[contains(@id, 'restaurant-card')]", visible=True)

    shop_num = 0
    # Loop through each element to find anchor tags and extract href attribute
    anchor_tags = await elements.querySelectorAll("a")
    for anchor_tag in anchor_tags:
        href = await (await anchor_tag.getProperty("href")).jsonValue()
        print(href)
        shop_num += 1
    print(f'There are {shop_num} shops')

    # Close the browser
    await browser.close()

# Set the desired geolocation coordinates (e.g., Taipei, Taiwan)
latitude = 22.610
longitude = 120.35

# Allow nested asyncio event loops
nest_asyncio.apply()

# Run the set_geolocation function to set the geolocation and perform scraping
asyncio.get_event_loop().run_until_complete(set_geolocation(latitude, longitude))


https://shop.ichefpos.com/store/ofsWPcfj/ordering?utm_source=safer_takeout&utm_medium=website&utm_campaign=safer_takeout&utm_id=safer_takeout_01
There are 1 shops
